## Naranja X 

In [1]:
import pandas as pd

df = pd.read_csv("../data/data_naranjax.csv")

df

,Unnamed: 0,Date,User,Tweet
0,1,2023-03-06 15:21:06+00:00,GeorgiGaveglio,@NaranjaX el tema no es pagarlo por el homeban...
1,5,2023-03-06 15:15:04+00:00,taru_commongirl,@NaranjaX no puedo registrarme en la app y aho...
2,7,2023-03-06 15:13:30+00:00,Shirsabi,@NaranjaX Si hablenme por privado por favor.
3,9,2023-03-06 15:10:47+00:00,JesoLedesma,@NaranjaX Ya les mande msj ! Ya me prometieron...
4,13,2023-03-06 15:04:58+00:00,smutsipou,@NaranjaX @ConsumidorPba Estoy esperando…
...,...,...,...,...
5817,9995,2022-12-08 15:22:14+00:00,YoaMrx,@NaranjaX Todavía no me llega la transferencia...
5818,9996,2022-12-08 15:21:11+00:00,juliantachu,@NaranjaX llevo más de 24 esperando que me lle...
5819,9997,2022-12-08 15:20:33+00:00,mariocarman,@NaranjaX Ese flequillo NO me da confianza.
5820,9998,2022-12-08 15:17:36+00:00,hearstprxx,@Franco70180993 @NaranjaX hice eso varias vece...


In [2]:
from pysentimiento import create_analyzer

analyzer = create_analyzer(task="emotion", lang="es")

analysis = analyzer.predict(df["Tweet"])

/home/jmperez/projects/twitter-brands/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 5822
  Batch size = 32
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
emotions = [a.output for a in analysis]

df["emotions"] = emotions

In [ ]:
pd.options.display.max_colwidth = 200

df

In [ ]:
df["emotions"].value_counts()

In [ ]:
from transformers import AutoModel, AutoTokenizer
from pysentimiento.preprocessing import preprocess_tweet
from datasets import Dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModel.from_pretrained("pysentimiento/robertuito-emotion-analysis")
tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-emotion-analysis")

model = model.to(device)

# Create dataset from pandas

dataset = Dataset.from_pandas(df)

In [ ]:
dataset = dataset.map(lambda ex: {"text": preprocess_tweet(ex["Tweet"], lang="es")}, batched=False)

In [ ]:
def _tokenize(batch):
    return tokenizer(
        batch["text"], padding=False, truncation=True
    )

dataset = dataset.map(_tokenize, batched=True, batch_size=64)

In [ ]:
# Create dataloader
from torch.utils.data import DataLoader

def collate_fn(examples):
    # Restrict to input_ids, attention_mask, and token_type_ids
    examples = [{k:v for k,v in ex.items() if k in ["input_ids", "attention_mask", "token_type_ids"]} for ex in examples]

    ret = tokenizer.pad(examples, padding=True)

    return ret
    
    


dataloader = DataLoader(dataset, batch_size=128, shuffle=False, collate_fn=collate_fn)

In [ ]:
import torch
from tqdm.auto import tqdm

outs = []

for batch in tqdm(dataloader):
    batch = {k:torch.LongTensor(v).to(device) for k,v in batch.items()}
    outs.append(model(**batch))